In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [2]:
import malaya
import malaya_speech
import tensorflow as tf

malaya.__version__, malaya_speech.__version__, tf.__version__

('4.6.1', '1.2.5', '1.15.5')

In [3]:
model = malaya.true_case.transformer(model = 'base', device = 'cpu:0')

In [4]:
aligner = malaya_speech.force_alignment.deep_ctc(model = 'hubert-conformer', device = 'cpu:0')

In [5]:
from glob import glob

wavs = glob('dari-pasentran-ke-istana-new/*.wav')
len(wavs)

6334

In [6]:
def put(alignment, length, min_threshold = 0.5, min_threshold_fullstop = 1.0):
    r = []
    for no, row in enumerate(alignment):
        if no > 0:
            if alignment[no]['start'] - alignment[no-1]['end'] >= min_threshold:
                r.append(',')

        r.append(row['text'])
    if length - alignment[-1]['end'] >= min_threshold_fullstop:
        r.append('.')
    else:
        r.append(',')
    return r

In [7]:
from tqdm import tqdm

results = {}
for w in tqdm(wavs):
    with open(w[:-3] + 'txt') as fopen:
        data = fopen.read().lower()
    
    y, sr = malaya_speech.load(w)
    result = aligner.predict(y, data)
    if ' '.join([r['text'] for r in result['words_alignment']]) != data:
        continue
    o = put(result['words_alignment'], len(y) / sr)
    o = ' '.join(o)
    o_true_case = model.greedy_decoder([o])[0]
    results[w] = {'output': o, 'output_true_case': o_true_case}

100%|██████████| 6334/6334 [1:02:28<00:00,  1.69it/s]


In [10]:
len(results)

6334

In [11]:
import json

with open('comma-dari-pasentran-ke-istana-sani.json', 'w') as fopen:
    json.dump(results, fopen)